# Set up

In [ ]:
# !git clone https://github.com/NLP-Reichman/2025_assignment_1.git
# !mv 2025_assignment_1/data data
# !rm 2025_assignment_1/ -r

# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import re
from itertools import product

is_using_colab = os.getenv("COLAB_RELEASE_TAG")
if is_using_colab:
    from google.colab import files

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [ ]:
from collections import defaultdict, Counter

def load_data():
    directory = "data"
    tweets = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
            language = os.path.splitext(filename)[0]
            raw_tweets = data.get("tweet_text", {})
            tweet_texts = clean_data(raw_tweets)
            for tweet_id, text in tweet_texts.items():
                tokens = tokenize_text(text)
                tweets.append([tokens, language])
    return np.array(tweets, dtype=object)

def tokenize_text(text):
    return list(text)

# TODO: Complete this function if necessary, or remove.
def clean_data(data):
    for tweet_id in data:
        text = data[tweet_id]
        # Remove retweets starting with RT @username:
        text = re.sub(r'^RT\s+@\w+:\s*', '', text)
        # Remove all mentions
        text = re.sub(r'@\w+', '', text)
        # Remove hyperlinks
        text = re.sub(r'http\S+', '', text)
        # Replace multiple whitespaces with a single space and trim
        text = re.sub(r'\s+', ' ', text).strip()
        data[tweet_id] = text
    return data

tweets = load_data()

def preprocess() -> list[str]:
    '''
    Return a list of characters, representing the shared vocabulary of all languages.
    Note: Here we assume "tweets" is a numpy ndarray where each row is [tweet_text, language].
    '''
    vocab_counter = Counter()
    for tweet_row in tweets:
        tweet_text = tweet_row[0]
        vocab_counter.update(tweet_text)
    vocab_counter.update('<start>')
    vocab_counter.update('<end>')
    vocab = list(vocab_counter.keys())
    return vocab

# We didnt want to change the function signature so we split it into separate helper function
def preprocess_one_lang(tweets, n) -> list[str]:
    '''
    Given a word-separated ndarray of size (N,), returns vocabulary for this ndarray
    Note: Here we assume all tweets are in the same language.
    
    n: the sentence length (for the n-gram)
    '''
    vocab_counter = Counter()
    processed_tweets = []
    for tweet_text in tweets:
        tweet_text = ['<start>'] * (n - 1) + tweet_text + ['<end>']
        processed_tweets.append(tweet_text)
        vocab_counter.update(tweet_text)
    vocab_counter.most_common()
    vocab = list(vocab_counter.keys())
    return vocab, processed_tweets

In [ ]:
vocab = preprocess()
vocab_size = len(vocab)
print(f"vocab length: {vocab_size}")
print(f"Some characters in the vocab: {vocab[:10]}")

## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1|V|$ to the LM if buiulding a smoothed LM.

In [ ]:
# TODO: fix process_one_lang to account for n-1 start
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :param smoothed: boolean indicating whether to apply smoothing
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    LM = {}
    
    # Filter tweets by the specified language
    tweet_texts = tweets[tweets[:, 1] == lang, 0]
    lang_vocab, processed_tweets = preprocess_one_lang(tweet_texts, n)
    lang_size = len(vocab)
    print(processed_tweets[0])
    if len(tweet_texts) == 0:
        raise ValueError('No such language!')
        
    lm_counts = defaultdict(Counter)
    for words_list in processed_tweets:
        # Iterate through each possible starting index for the n-gram
        for i in range(len(words_list) - (n - 1)):
            # Generate the key as a concatenated string of n-1 words
            key = ''.join(words_list[i:i + n - 1])
            next_word = words_list[i + n - 1]
            lm_counts[key].update(next_word)

    for key in lm_counts:
        total = sum(lm_counts[key].values())
        prob_dict = {}
        
        if smoothed:
            # Apply add-1 (Laplace) smoothing
            total_smoothed = total + lang_size
            for word in lang_vocab:
                count = lm_counts[key].get(word, 0) + 1
                prob = count / total_smoothed
                prob_dict[word] = prob
        else:
            # No smoothing, use observed frequencies
            for word, count in lm_counts[key].items():
                prob_dict[word] = count / total
        LM[key] = prob_dict
    return LM

def print_sorted_counter_dict(counter_dict):
    """
    Prints a defaultdict(Counter) preserving original key order,
    with each Counter's contents sorted by value (descending) then key (ascending).
    
    Args:
        counter_dict (defaultdict): A defaultdict where values are Counters
    """
    for key in counter_dict:  # Iterate in original insertion order
        counter = counter_dict[key]
        # Get items sorted by value (descending), then by key (ascending)
        sorted_items = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
        print(f"{key}:")
        for item, count in sorted_items:
            print(f"  {item}: {count}")

In [ ]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

In [ ]:
print_sorted_counter_dict(LM)

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [ ]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  pp = 0.0
  for i in range(len(text) - (n - 1)):
    ngram = ''.join(text[i:i + n - 1])
    ngram_prob = model.get(ngram).get(text[i + n - 1], 0.0)
    pp += np.log(ngram_prob)
  pp = (- pp / len(text))
  return pp

In [ ]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  pp = 0
  tweet_texts = tweets[tweets[:, 1] == target_lang, 0]
  _, processed_tweets = preprocess_one_lang(tweet_texts, n)
  joined_text = [char for sentence in processed_tweets for char in sentence] # merges all tweets in the language into a single large sentence
  pp = perplexity(model, joined_text, n)
  return pp


In [ ]:
LM = build_lm("en", 3, True)

In [ ]:
eval(LM, 'en', 3)

print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


In [ ]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on differnet n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [ ]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  df  = pd.DataFrame()
  ########################################
  # your code

  ########################################
  return df


## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [ ]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = ""
  ########################################
  # your code

  ########################################
  return text

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [ ]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [ ]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
if is_using_colab:
    files.download('results.json')
########################################

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l